In [7]:
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic
from astropy.io import fits

In [6]:
hp = HEALPix(nside=256, frame = Galactic())
npix = hp.npix
lon, lat = hp.healpix_to_lonlat(range(npix))  # gives galactic longitude and latitude (in degrees) of each healpix pixel

In [8]:
hdu1 = fits.open('whsky001.fit')
NAXIS1 = hdu1[0].header['NAXIS1']
NAXIS2 = hdu1[0].header['NAXIS2']
NAXIS3 = hdu1[0].header['NAXIS3']
CRVAL1 = hdu1[0].header['CRVAL1']
CDELT1 = hdu1[0].header['CDELT1']
CRPIX1 = hdu1[0].header['CRPIX1']
CROTA1 = hdu1[0].header['CROTA1']
CRVAL2 = hdu1[0].header['CRVAL2']
CDELT2 = hdu1[0].header['CDELT2']
CRPIX2 = hdu1[0].header['CRPIX2']
CROTA2 = hdu1[0].header['CROTA2']

FileNotFoundError: [Errno 2] No such file or directory: 'whsky001.fit'